In [1]:
%pwd

'c:\\Users\\Vishwamohan\\OneDrive\\Desktop\\Vishu\\Projects\\CNN_Classifier\\research'

In [2]:
# setting location so that we can import CNNClassifier module
import sys
import os

# Navigate two levels up to the project root directory (adjust if necessary)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))

# Add the project root to sys.path
sys.path.append(project_root)

# Now, the CNNClassifier module should be found
project_root

'c:\\Users\\Vishwamohan\\OneDrive\\Desktop\\Vishu\\Projects\\CNN_Classifier\\src'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float

In [4]:
from CNNClassifier.constants import *
from CNNClassifier.utils.utils import read_yaml, create_dir
import tensorflow as tf

c:\Users\Vishwamohan\anaconda new\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[2024-10-13 00:42:05,202: INFO: utils]: NumExpr defaulting to 4 threads.


In [5]:
class ConfigurationManager:
    def __init__(
        self,config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_dir([self.config.artifacts_root])
        
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "PetImages")
        create_dir([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE
        )

        return training_config 

In [6]:
os.chdir('../')

In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [8]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

        # Compile the model after loading to reset optimizer and other settings
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss='categorical_crossentropy',  # Use 'binary_crossentropy' if it's binary classification
            metrics=['accuracy']
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [9]:
from scipy import ndimage

In [10]:
try:
    config = ConfigurationManager()
  
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-10-13 00:42:06,408: INFO: utils]: create directory at : artifacts
[2024-10-13 00:42:06,410: INFO: utils]: create directory at : artifacts\training
[2024-10-13 00:42:07,062: WARNING: saving_utils]: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


c:\Users\Vishwamohan\anaconda new\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 144/1250 ━━━━━━━━━━━━━━━━━━━━ 1:06:10 4s/step - accuracy: 0.6967 - loss: 3.5810

c:\Users\Vishwamohan\anaconda new\lib\site-packages\PIL\TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7890 - loss: 2.8738

KeyboardInterrupt: 